In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import requests
import pandas as pd
import plotly.express as px

# Initialize Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("New York Crime Data Dashboard"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in range(2001, 2024)] + [{'label': 'Select All', 'value': 'all'}],
        value=['2023'],
        multi=True,
        style={'width': '50%'}
    ),
    # Other dropdowns...
    dcc.Dropdown(
        id='borough-dropdown',
        options=[{'label': borough, 'value': borough} for borough in ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']] + [{'label': 'Select All', 'value': 'all'}],
        value=['MANHATTAN'],
        multi=True,
        style={'width': '50%'}
    ),
    dcc.Dropdown(
        id='offense-dropdown',
        options=[{'label': offense, 'value': offense} for offense in [
            'VIOLENT CRIMES', 'DRUG-RELATED OFFENSES', 'ASSAULT', 'THEFT',
            'CRIMINAL TRESPASS', 'SEXUAL OFFENSES', 'ADMIN AND STATE LAWS', 
            'WEAPONS', 'PUBLIC ORDER', 'FRAUD AND FORGERY', 'CRIMINAL MISCHIEF & RELATED OF',
            'PUBLIC SAFETY AND MORALITY', 'TRAFFIC OFFENSES', 'HARRASSMENT 2', '(null)',
            'OTHER OFFENSES RELATED TO THEF', 'MISC OFFENSES', 'KIDNAPPING & RELATED OFFENSES', 
            'LOITERING/GAMBLING (CARDS, DIC', 'CRIMINAL MISCHIEF & RELATED OFFENSES', 'HARASSMENT',
            'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)'
        ]] + [{'label': 'Select All', 'value': 'all'}],
        value=['VIOLENT CRIMES'],
        multi=True,
        style={'width': '50%'}
    ),
    # Other dropdowns...
    dcc.Dropdown(
        id='ethnicity-dropdown',
        options=[{'label': ethnicity, 'value': ethnicity} for ethnicity in [
            'BLACK', 'WHITE', 'WHITE HISPANIC', 'BLACK HISPANIC',
            'ASIAN / PACIFIC ISLANDER', 'UNKNOWN', 'OTHER',
            'AMERICAN INDIAN/ALASKAN NATIVE'
        ]] + [{'label': 'Select All', 'value': 'all'}],
        value=['BLACK'],
        multi=True,
        style={'width': '50%'}
    ),
    dcc.Dropdown(
        id='gender-dropdown',
        options=[{'label': gender, 'value': gender} for gender in ['M', 'F', 'U']] + [{'label': 'Select All', 'value': 'all'}],
        value=['M'],
        multi=True,
        style={'width': '50%'}
    ),
    dcc.Dropdown(
        id='age-category-dropdown',
        options=[{'label': age, 'value': age} for age in [
            '25-44', '45-64', '18-24', '<18', '65+'
        ]] + [{'label': 'Select All', 'value': 'all'}],
        value=['25-44'],
        multi=True,
        style={'width': '50%'}
    ),
    html.Button('Apply', id='apply-button', n_clicks=0),
    dcc.Graph(id='crime-graph'),
    html.Div(id='output-container')
])

# Define the callback to update data based on selected filters
@app.callback(
    Output('crime-graph', 'figure'),
    [Input('apply-button', 'n_clicks')],
    [
        State('year-dropdown', 'value'),
        State('borough-dropdown', 'value'),
        State('offense-dropdown', 'value'),
        State('ethnicity-dropdown', 'value'),
        State('gender-dropdown', 'value'),
        State('age-category-dropdown', 'value')
    ]
)
def update_output(n_clicks, selected_years, selected_boroughs, selected_offenses, selected_ethnicities, selected_genders, selected_age_categories):
    if n_clicks > 0:
        # Replace 'all' with actual values if 'Select All' is chosen
        if 'all' in selected_years:
            selected_years = list(range(2001, 2024))
        if 'all' in selected_boroughs:
            selected_boroughs = ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']
        if 'all' in selected_offenses:
            selected_offenses = [
                'VIOLENT CRIMES', 'DRUG-RELATED OFFENSES', 'ASSAULT', 'THEFT',
                'CRIMINAL TRESPASS', 'SEXUAL OFFENSES', 'ADMIN AND STATE LAWS', 
                'WEAPONS', 'PUBLIC ORDER', 'FRAUD AND FORGERY', 'CRIMINAL MISCHIEF & RELATED OF',
                'PUBLIC SAFETY AND MORALITY', 'TRAFFIC OFFENSES', 'HARRASSMENT 2', '(null)',
                'OTHER OFFENSES RELATED TO THEF', 'MISC OFFENSES', 'KIDNAPPING & RELATED OFFENSES', 
                'LOITERING/GAMBLING (CARDS, DIC', 'CRIMINAL MISCHIEF & RELATED OFFENSES', 'HARASSMENT',
                'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)'
            ]
        if 'all' in selected_ethnicities:
            selected_ethnicities = [
                'BLACK', 'WHITE', 'WHITE HISPANIC', 'BLACK HISPANIC',
                'ASIAN / PACIFIC ISLANDER', 'UNKNOWN', 'OTHER',
                'AMERICAN INDIAN/ALASKAN NATIVE'
            ]
        if 'all' in selected_genders:
            selected_genders = ['M', 'F', 'U']
        if 'all' in selected_age_categories:
            selected_age_categories = ['25-44', '45-64', '18-24', '<18', '65+']

        # Send request to backend
        response = requests.post("http://backend:5000/filter", json={
            "years": selected_years,
            "boroughs": selected_boroughs,
            "offenses": selected_offenses,
            "ethnicities": selected_ethnicities,
            "genders": selected_genders,
            "age_categories": selected_age_categories
        })
        data = pd.DataFrame(response.json())
        
        # Example visualization
        fig = px.scatter(data, x='date', y='crime_count', color='crime_type', title='Crime Data for Selected Filters')
        
        return fig
    return px.scatter(title="Select filters and click Apply")

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0')
